In [110]:
import os
import configparser
from datetime import datetime

from twython import Twython
import pandas as pd

Matplotlib is building the font cache using fc-list. This may take a moment.


In [22]:
# const
CRED_FNAME = 'credentials.txt'

In [111]:
class TweetLoader(object):
    
    def __init__(self, cred_dir = '../twitter-cred', cred_fname = CRED_FNAME):
        cred_fpath = os.path.join(cred_dir, cred_fname)
        print(f"Retrieve twitter credential: {cred_fpath}")
        if os.path.exists(cred_fpath):
            config = configparser.ConfigParser()
            config.read(cred_fpath)
            try:
                self._app_key = config['OAUTH']['app_key']
                self._app_secret = config['OAUTH']['app_secret']
                self._oauth_token = config['OAUTH']['oauth_token']
                self._oauth_token_secret = config['OAUTH']['oauth_token_secret']
            except KeyError as ex:
                raise ValueError(f"Failure to read field: {ex}")
                
            self.twitter = Twython(self._app_key, self._app_secret)
        else:
            raise ValueError(f"Failed to read credential file: {cred_fpath}")
            
    def get_tweets(self, query, from_date, to_date, count=50):
        tweets = []
        results = self.twitter.search(q=query, until=to_date, count=count)
        
        to_date = datetime.strptime(to_date, "%Y-%m-%d")
        from_date = datetime.strptime(from_date, "%Y-%m-%d")
        for result in results['statuses']:
            tweet_date = datetime.strptime(result['created_at'], "%a %b %d %H:%M:%S %z %Y").replace(tzinfo=None)
            if tweet_date < from_date:
                break
            tweet_text = result['text']
            tweets.append(tweet_text)
            
        return tweets

In [113]:
# Usage: 
# loader = TweetLoader()
# loader.get_tweets("$AAPL", "2019-12-07", "2019-12-08", 10)

Retrieve twitter credential: ../twitter-cred/credentials.txt
2019-12-07 23:59:21
2019-12-07 23:59:21
2019-12-07 23:59:20
2019-12-07 23:59:20
2019-12-07 23:59:19
2019-12-07 23:59:19
2019-12-07 23:59:19
2019-12-07 23:54:23
2019-12-07 23:53:17
2019-12-07 23:53:12


['RT @vieiraUAE: Goldman Sachs Biggest Scandal to Date Bursts! Apple Victim of Stock Market Manipulation $GS $AAPL $QQQ https://t.co/U7DpFNZ5…',
 'RT @TraderRobertH: $AAPL from 2019-12-07. Gap up, took out prior resistance, re-test, off to the races. Aided by strong $SPY for a power tr…',
 "RT @MC2085: $AAPL Weekly Chart.\n\nPerfectly Normal Candle.\n\nNice Volume off the August $192.58 Low. (Note to Siri: That's Sarcasm). https://…",
 'RT @Celina1422: @Sandro_power Fantastic gains in $AAPL and $GOOGL, watch the video and find out more!',
 'RT @Sandro_power: We had a great week with just  2 powerful plans we focused on $AAPL and GOOGL with skill right after dec 03 This Video  l…',
 'RT @pineconemacro: $AAPL worth more than energy...all of it https://t.co/P9Yv8LgpAc',
 "RT @TEEELAZER: 9x since Halloween buyin🍎 $AAPL after it took out the low of previous day would've worked, so don't let someone tell you it'…",
 'RT @Sandro_power: We had a great week with just  2 powerful plans we focuse